In [1]:
import libmata as mata
from libmata.nfa.nfa import *
from libmata.parser import *
import libmata.plotting

## Auxiliary Functions

In [2]:
def distance_from(aut: Nfa) -> dict[int,int]:
    queue = []
    dist = dict()
    for q in aut.initial_states:
        dist[q] = 0
        queue.append(q)

    while len(queue) != 0:
        act = queue.pop(0)
        for trans in aut.get_trans_from_state_as_sequence(act):
            if not trans.target in dist:
                dist[trans.target] = dist[act] + 1
                queue.append(trans.target)
    return dist

def hexa_heat(minimum, maximum, value):
    ratio = 2 * (value-float(minimum)) / float((maximum - minimum))
    b = int(max(0, 255 * (1 - ratio)))
    r = int(max(0, 255 * (ratio - 1)))
    g = 255 - b - r
    return "#{:02x}{:02x}{:02x}".format(r, g, b)

Creation of example automata

In [3]:
libmata.plotting.store()['alphabet'] = mata.alphabets.OnTheFlyAlphabet.from_symbol_map({"0":0, "1":1})
lhs = divisible_by(4)
lhs.label = "aut1"
rhs = divisible_by(3)
rhs.label = "aut2"
result = concatenate(lhs, rhs)
result.label = "aut1.aut2"

## Conditional State/Transition Formatting

In [4]:
e_h = [
    (lambda aut, edge: edge.symbol == 1, {'color': 'grey'}),
    (lambda aut, edge: edge.symbol == 0, {'style': 'dashed', 'color': 'grey'})
]
n_h = [
    (lambda aut, state: state in aut.final_states, {'color': 'red', 'fillcolor': 'red'}),
    (lambda aut, state: state in aut.initial_states, {'color': 'green', 'fillcolor': 'green'}),
]
libmata.plotting.plot(lhs, rhs, result, node_highlight=n_h, edge_highlight=e_h)

## SCC Visualization

In [5]:
e_h = [
    (lambda aut, edge: edge.symbol == 1, {'color': 'grey'}),
    (lambda aut, edge: edge.symbol == 0, {'style': 'dashed', 'color': 'grey'})
]
n_h = [
    (lambda aut, state: state in aut.final_states, {'color': 'red', 'fillcolor': 'red'}),
    (lambda aut, state: state in aut.initial_states, {'color': 'green', 'fillcolor': 'green'}),
]
libmata.plotting.plot(lhs, rhs, result, with_scc=True, node_highlight=n_h, edge_highlight=e_h)

## Example of a Heatmap

In [6]:
heat_map = distance_from(result)
max_heat = max(heat_map.values())
e_h = [
    (lambda aut, edge: edge.symbol == 1, {'color': 'gray'}),
    (lambda aut, edge: edge.source == edge.target, {'style': 'dashed', 'color': 'red'})
]
n_h = [
    (lambda aut, state, j=i: state == j, libmata.plotting.Style.filled(hexa_heat(0, max_heat, heat_map[i]))) for i in range(0,result.num_of_states())
]
libmata.plotting.plot(result, node_highlight=n_h, edge_highlight=e_h)

## Regex Example

In [7]:
from libmata import nfa, alphabets, parser, plotting
aut1 = parser.from_regex('((a+b)*a)*')
aut2 = parser.from_regex('aab*')
con_aut = nfa.nfa.concatenate(aut1, aut2).trim()
# Setting correct alphabet.
plotting.store()['alphabet'] = \
    alphabets.OnTheFlyAlphabet.from_symbol_map({'a':97, 'b':98})
# Plot the concatenated and trimmed automaton.
e_h = [
    (lambda aut, e: e.symbol == 98, {'color': 'black'}),
    (lambda aut, e: e.symbol == 97, {'style': 'dashed', 'color': 'black'})
]
n_h = [
    (lambda aut, q: q in aut.final_states, {'color': 'red', 'fillcolor': 'red'}),
    (lambda aut, q: q in aut.initial_states, 
        {'color': 'orange', 'fillcolor': 'orange'}),
]
plotting.plot(con_aut, with_scc=True, node_highlight=n_h, edge_highlight=e_h)